# Progetto data Analytics

## Installing libraries

In [1]:
#for install libraries
!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain

Reading package lists... Done
Building dependency tree       
Reading state information... Done
build-essential is already the newest version (12.4ubuntu1).
python-dev is already the newest version (2.7.15~rc1-1).
zlib1g-dev is already the newest version (1:1.2.11.dfsg-0ubuntu2).
zlib1g-dev set to manually installed.
libxml2 is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
libxml2 set to manually installed.
libxml2-dev is already the newest version (2.9.4+dfsg1-6.1ubuntu1.2).
The following package was automatically installed and is no longer required:
  libnvidia-common-410
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 6 not upgraded.
     |████████████████████████████████| 71kB 2.7MB/s 
  Stored in directory: /root/.cache/pip/wheels/e7/5d/6f/fc3c2364dfd3c4cfd15d786b156077c52209d9af45496fdf12
Successfully built cairocffi
     |████████████████████████████████| 378kB 2.8MB/s 
  Stored in directory: /root/.cache/pip/wheels/41/d6/02/34eeba

#### Execute this only if you are Alex Caselli on Colab

In [12]:
from google.colab import drive
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np
drive.mount('/content/gdrive')
baseURL = '/content/gdrive/My Drive/Data Analytics/';

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Import Libraries and set local path

In [0]:
import networkx as nx
import igraph as ig
import louvain
from collections import OrderedDict
import numpy as np

baseURL = '../'

## Creation of the Total Graph and the Alliance Graph
You don't have to run this code if you already have all the graph files in your dataset


In [0]:
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()

attackSelf = 0
messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, nGraph):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    realAttack = nx.MultiDiGraph()
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
    attackSelf = 0
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message", time = itr[3]['edgetime'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label = "Trade", time = itr[3]['edgetime'])

    #for itr in attackGraph.edges_iter(None, True, True):
    for itr in list(attackGraph.edges(None, True, True)):
        if int(itr[0]) != int(itr[1]):
            realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack", time = itr[3]['edgetime'])
        else:
            attackSelf += 1
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    totalGraph = nx.compose(allianceGraph, realAttack)
    # save total graph and alliance graph in the correct file
    nx.write_graphml(totalGraph, baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    nx.write_graphml(allianceGraph, baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
    print("ATTACK GRAPH: ", nx.info(attackGraph))
    print("NO SENSE SELF ATTACK EDGE: ", attackSelf)
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") 

In [0]:
def weightCompute(scoreList):
    if scoreList[0] > 0 and scoreList[1] > 0:
        return 2*sum(scoreList)
    else:
        return sum(scoreList)

##TRANSFORMATION OF THE ALLIANCE MULTIGRAPH TO A PLAIN GRAPH WITH WEIGHTED EDGES

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:
#Creation of 30 plain graphs
nGraph = 30
realAllianceGraph = nx.DiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0

#for each day do
for i in range(0, nGraph):
    
    #Create a new plain graph and read the multigraph for that day
    allianceGraph = nx.MultiDiGraph()
    realAllianceGraph = nx.DiGraph()
    checkGraph = ig.Graph()
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    
    #initialize the counters
    tradeCount = 0
    messageCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    
    #for each edge of the multigraph
    for itr in list(allianceGraph.edges(None, True, True)):
        # print(itr)
        #create a tmp dictionary to count the numbers of messages and trades between each couple of nodes
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
            else:
                
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tradeCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tradeCount += 1

    #For each edges saved in the dictionary add it to the plain graph
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), Weight = weightCompute(edgesDict[fromVertex][toVertex]))
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

        
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allianceGraph))

    #Count number of inactive nodes
    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(realAllianceGraph))
    print("")
    # for itr in realAllianceGraph.edges_iter(None, True, True):
        # print(itr)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    nx.write_graphml(realAllianceGraph, baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

##TRANSFORMATION OF THE TOTAL MULTIGRAPH TO A MULTIGRAPH  WITH WEIGHTED EDGES FOR EACH TYPE OF INTERACTION

---

Don't run this code if you already have the plain graph in your dataset folder

In [0]:

nGraph = 30
compactAllGraph = nx.MultiDiGraph()


checkGraph = ig.Graph()
edgesDict = dict()

tradeCount = 0
messageCount = 0
attackCount = 0
finalCheck = 0

#for each day 
for i in range(0, nGraph):
    
    #create new plaingraphs 
    allGraph = nx.MultiDiGraph()
    compactAllGraph = nx.MultiDiGraph()
    checkGraph = ig.Graph()
    allGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    checkGraph = checkGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    
    #initialize counters
    tradeCount = 0
    messageCount = 0
    attackCount = 0
    sumOfVertex = 0
    nStep = 0
    zeroOutDegreeCont = 0
    finalCheck = 0
    edgesDict = dict()

    print("DAY: ", (i+1))
    print("START GRAPH")
    print(nx.info(allGraph))
    print("")

    #create a tmp dictionary to count the numbers of messages, trades and attacks between each couple of nodes 
    for itr in list(allGraph.edges(None, True, True)):
        # print(itr)
        if itr[0] in edgesDict:
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]][2] += 1
                    attackCount += 1
            else: 
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                    tradeCount += 1
                elif itr[3]['Label'] == 'Attack':
                    edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                    attackCount += 1
        else:
            edgesDict[itr[0]] = {}
            
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0, 0]
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1, 0]
                tradeCount += 1
            elif itr[3]['Label'] == 'Attack':
                edgesDict[itr[0]][itr[1]] = [0, 0, 1]
                attackCount += 1

   #For each edges saved in the dictionary add it to the plain graph with the correct label 
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if edgesDict[fromVertex][toVertex][0] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="m", Label = "Message", Weight = edgesDict[fromVertex][toVertex][0])
                nStep += 1
            if edgesDict[fromVertex][toVertex][1] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="t", Label = "Trade", Weight = edgesDict[fromVertex][toVertex][1])
                nStep += 1
            if edgesDict[fromVertex][toVertex][2] > 0:
                compactAllGraph.add_edge(int(fromVertex), int(toVertex), key="a", Label = "Attack", Weight = edgesDict[fromVertex][toVertex][2])
                nStep += 1
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            sumOfVertex += sum(edgesDict[fromVertex][toVertex])

      
    print("FROM VERTICES NUMBER IN DICTIONARY: ", len(edgesDict))
    print("START GRAPH ALL VERTICES: ", len(allGraph))

    for vertex in checkGraph.vs:
        if vertex.outdegree() == 0:
            zeroOutDegreeCont += 1
    print("VERTICES WITH OUTDEGREE 0:", zeroOutDegreeCont)
    print("")

    print("END GRAPH")
    print(nx.info(compactAllGraph))
    print("")
    
    #Crazy things of Villa
    for itr in list(compactAllGraph.edges(None, True, True)):
        finalCheck += itr[3]['Weight']
    print("PARANOIC CHECK: ", finalCheck)
    print("SUM OF WEIGHT BEFORE MULTIPLY (START GRAPH EDGES NUMBER): ", sumOfVertex)
    print("NUMBER OF STEP (END GRAPH EDGES NUMBER): ", nStep)
    print("NUMBER OF TRADE EDGES MET: ", tradeCount)
    print("NUMBER OF MESSAGE EDGES MET: ", messageCount)
    print("NUMBER OF ATTACK EDGES MET: ", attackCount)
    nx.write_graphml(compactAllGraph, baseURL + "Dataset/Graphml-File/compactAllGraph/cAllGraph-2009-12-"+str(i+1)+".graphml")
    print('----------------------------------------------------------------------')

    

##Creation of a dictionaty for maintain all nets
Create a dictionary of lists, the key rappresents the day (start 0 end 29), the list is a list of nets about the specific day with this order: (attackGraph, messageGraph, TradeGraph, allianceGraph, allGraph, compactallGraph)

In [0]:
#if i can't read from Drive is caused by this
!ls '/content/gdrive/My Drive/Data Analytics/Dataset (1)'

# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()
compactAllGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    compactAllGraph = compactAllGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/compactAllGraph/cAllGraph-2009-12-"+str(i+1)+".graphml")
    
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph, compactAllGraph) 

Duplicate per Alex

In [29]:
#if i can't read from Drive is caused by this
!ls '/content/gdrive/My Drive/Data Analytics/Dataset (1)'

# our structure 
allNetwork = dict()

nGraph = 30

# for structure graphs inserting
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()
totalGraph = ig.Graph()
allianceGraph = ig.Graph()
compactAllGraph = ig.Graph()

for i in range(0, nGraph):
    print("Day: ", (i + 1))
    
    # read all needed graphs
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    compactAllGraph = compactAllGraph.Read_GraphML(baseURL + "Dataset (1)/Graphml-File/compactAllGraph/cAllGraph-2009-12-"+str(i+1)+".graphml")
    
    
    # put as a list in a dictionary 
    allNetwork[i] = (attackGraph, messageGraph, tradeGraph, allianceGraph, totalGraph, compactAllGraph) 

Communities  CSV-Files	Graphml-File  readme.txt
Day:  1
Day:  2
Day:  3
Day:  4
Day:  5
Day:  6
Day:  7
Day:  8
Day:  9
Day:  10
Day:  11
Day:  12
Day:  13
Day:  14
Day:  15
Day:  16
Day:  17
Day:  18
Day:  19
Day:  20
Day:  21
Day:  22
Day:  23
Day:  24
Day:  25
Day:  26
Day:  27
Day:  28
Day:  29
Day:  30


### This below cell will be use for any kind of check about nets, variables, indexes; for this reason no one comments are required

In [0]:
for i in range(0, len(allNetwork)):
    for vertex in allNetwork[i][5].vs:
        if vertex['id'] == '12027':
            print("{:>20}{:>3}".format("PRESENT IN THE DAY:", str(i + 1)))
            print("{:>20}{:>3}".format("DAY INDEGREE: ", vertex.indegree()))
            print("{:>20}{:>3}".format("DAY OUTDEGREE: ", vertex.outdegree()))
            for edge in allNetwork[i][5].es:
                if allNetwork[i][5].vs[edge.tuple[0]]['id'] == '12027':
                    print("{:>20}{:>3}".format("OUTDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))
                if allNetwork[i][5].vs[edge.tuple[1]]['id'] == '12027':
                    print("{:>20}{:>3}".format("INDEGREE LABEL: ", edge['Label']))
                    print("{:>20}{:>3}".format("QUANTITY: ", edge['Weight']))    
            print("") 
    
'''for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[0]]['id'] == '2789':
            print(edge)
print("")
for i in range(0, len(allNetwork)):
    print("DAY: "+str(i+1))
    for edge in allNetwork[i][4].es:
        if allNetwork[i][4].vs[edge.tuple[1]]['id'] == '2789':
            print(edge)'''


# 11850

### Domande da fare
**1)** Il numero massimo e minimo di nodi, dato il paper, è corretto e ci torna; il numero minimo e massimo di archi invece no. Il fatto è molto strano perché noi semplicemente apriamo il file graphml e stampiamo le info associate. Lavoriamo facendo finta di nulla? <br>
1) è stato detto di fregarsene, l'importante è che si usi o il file gml oppure il csv, non bisogna usare entrambi

**2)** Abbiamo osservato come, dato il file di community e la rete totale (attack + trade + message), non tutti i nodi sono presenti in questo file. Indagando su questi abbiamo poi osservato che subiscono (nella maggior parte dei casi) attacchi. Possiamo quindi dedurre che si tratti effettivamente di nodi non affiliati a nessuna alleanza in quanto utenti che hanno smesso di giocare o stanno iniziando a giocare (alcuni non subiscono attacchi ma commerciano e attaccano, scudo prottetivo 30 giorni, punteggio minimo giocare)? <br>
2) è normale in quanto le community segnate sono quelle registrate dagli utenti stessi sul server, quindi è possibile che alcuni nodi non siano affiliati a nessuna alleanza

**3)** Le misure di centralità sono possibili per reti multigrafo? <br>
**4)** In cosa differisce la presentazione dalla demo? <br>
**5)** Le community di dimensione minore di 10 come possiamo tratterle in maniera intelligente? Le associamo tutte a uno stesso identificativo, le eliminiamo, che facciamo?


# START ANALYSIS !!!!

### Users number trend in the 30 days

In [0]:
def userNumberTrend(allNetwork):
    nUsers = [0]*len(allNetwork)
    for i in range(0, len(allNetwork)):
        nUsers[i] = allNetwork[i][4].vcount()

    print("Users in the 30 days: ", nUsers)
    print("")
    print("{:>53}{:>7}.".format("Average users number in the 30 days is: ", round(sum(nUsers)/len(nUsers)), 2))
    print("{:>53}{:>7}.".format("The maximum number of users was recorded on the day: ", nUsers.index(max(nUsers))+1))
    print("")

### Edges number trend in the 30 days

In [0]:
def edgeNumberTrend(allNetwork):  
    nAttack = [0]*len(allNetwork)
    nTrade = [0]*len(allNetwork)
    nMessage = [0]*len(allNetwork)

    for i in range(0, nGraph):
        for edge in allNetwork[i][4].es:
            if edge['Label'] == "Attack":
                nAttack[i] += 1
            elif edge['Label'] == "Trade":
                nTrade[i] += 1
            elif edge['Label'] == "Message":
                nMessage[i] += 1
            else:
                print("This is a problem...")

    print("{:>33}".format("Number of attacks in the 30 days:"), end = " ")
    print(nAttack)
    print("")
    print("{:>32}".format("Number of trades in the 30 days:"), end = " ")
    print(nTrade)
    print("")
    print("{:>34}".format("Number of messages in the 30 days:"), end = " ")
    print(nMessage)
    print("")
    print("{:>40}{:>9}".format("Average number of attacks in 30 days:", sum(nAttack)/len(nAttack)))
    print("{:>40}{:>9}".format("Average number of trades a in 30 days:", round(sum(nTrade)/len(nTrade), 1)))
    print("{:>40}{:>9}".format("Average number of messages a in 30 days:", round(sum(nMessage)/len(nMessage), 1)))  
    print("")
    print("{:>40}{:>9}".format("Day with the highest number of attacks:", nAttack.index(max(nAttack))+1))
    print("{:>40}{:>9}".format("Day with the highest number of trades:", nTrade.index(max(nTrade))+1))
    print("{:>40}{:>9}".format("Day with the highest number of messages:", nMessage.index(max(nMessage))+1)) 
    print("")

Code for generate a csv with edge and nodes trends in 30days

In [0]:
nAttack, nTrade, nMessage=edgeNumberTrend(allNetwork)
nUsers=userNumberTrend(allNetwork)


#write trend in a CSV
with open(baseURL+'Dash/userAndEdge_trends_30days.csv', 'w') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['day', 'users', 'attack', 'trade', 'message'])

    for i in range(0, 30):
        filewriter.writerow([i, nUsers[i],nAttack[i], nTrade[i], nMessage[i]])

### Trickster detection tool 
This tool is designed for identifying dishonest users who manage two accounts and use one for the main account. The assumption is: if i see you two times do more trading than any else action you are a suspected and is necessary to see what are you done in the period (30 days). 

In [0]:
# Main function call all function that compose the tool
def tricksterAnalysis(allNetwork):
    
    # make a list with main suspect for every day, in the end we have a following structure
    #{DAY: {idSuspected: {idBeneficiry: qta}}} 
    # a dictionary where for first external key we have a day, inside 1 lvl key is suspected id, in the last 2 lvl
    # a dictionary of beneficiary key with quantity of trading
    dailySuspicions = dict()
    print("Start trickester Analysis")
    print("Watching for dishonest user in the period...")
    for i in range(0, len(allNetwork)):
        # print("DAY: ", str(i+1))
        dailySuspicions[i] = tricksterDetectionTool(allNetwork[i][4])
    
    # from first skimming compute a guilty index and print the player with more then 0.3 on index
    edgesCount(printRealSuspected(dailySuspicions, len(dailySuspicions)), allNetwork)

# create a day dictionary structur (see below)
def tricksterDetectionTool(totalNet):
    zeroIndegreeVertex = list()

    # get, specific net, all vertex with 0 indegree
    for vertex in totalNet.vs:
        if vertex.indegree() == 0:
            zeroIndegreeVertex.append(vertex)
    
    '''for vertex in zeroIndegreeVertex:
        print(vertex)
        print(vertex.indegree())
        print(vertex.outdegree())'''
    #print(zeroIndegreeVertex)
    
    # create a dictionary for keeping suspects with respective beneficiary
    suspecteds = dict()
    beneficiary = dict()
    key = ""
    
    # for every zero indegree vertex count the number of edge and the number of trade edge
    # if find a trade edge save the beneficiary of this trade and quantity
    for vertex in zeroIndegreeVertex:
        nEdges = 0
        nTrades = 0
        key = ""
        beneficiary = dict()
        for edge in totalNet.es:
            if totalNet.vs[edge.tuple[0]]['id'] == vertex['id']:
                nEdges += 1
                if edge['Label'] == 'Trade':
                    # print(edge)
                    # print(totalNet.vs[edge.tuple[0]]['id'])
                    # print(vertex['id'])
                    if totalNet.vs[edge.tuple[1]]['id'] in beneficiary:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] += 1
                    else:
                        beneficiary[totalNet.vs[edge.tuple[1]]['id']] = 1
                    nTrades += 1
        # if the trades edges number is more than 95%, the node is suspected, saved it!
        if nTrades/nEdges >= 0.95:
            # print(nEdges)
            # print(nTrades)
            key = str(vertex['id'])
            # print(key)
            # print(beneficiary)
            suspecteds[key] = beneficiary
    
    # for key in suspecteds:
    #    print(key, suspecteds[key])
    return suspecteds    

# from all day suspects we want a more compact structure for see what happen to our suspects node in the period
# (30 days). The structure is a dictionary: 
# {idSuspected: [nWatchTimes, {idBeneficiary: [nWatchTimes, qnt]}, nWatchSuspectAsBeneficiary, qnt]}
# from this we can delete from suspects list the node with only one appearance
def printRealSuspected(allSuspects, nDays):
    
    print("Compact recovered information...")
    beneficiaries = dict()
    suspectsSummary = dict()
    finalSuspects = dict()
    
    
    for day in allSuspects:
        for suspect in allSuspects[day]:
            # print("Suspected: ", suspect)
            # increase the number of nWatchTimes about suspects
            if str(suspect) in suspectsSummary:
                suspectsSummary[suspect][0] += 1
            else:
                suspectsSummary[suspect] = list([1, dict(), 0, 0])
                suspectsSummary[suspect][0] = 1
                # print(suspectsSummary[suspect][0])
            # print("TIME: ", suspectsSummary[suspect])
            
            # for all suspect beneficiary increase nWatchTimes about beneficiary and qnt about trade
            for beneficiary in allSuspects[day][suspect]:
                # print("Beneficiary is: ", beneficiary)
                # print("Benefit is: ", allSuspects[day][suspect][beneficiary])
                # print(str(beneficiary) in suspectsSummary[suspect][1])
                if str(beneficiary) in suspectsSummary[suspect][1]:
                    suspectsSummary[suspect][1][beneficiary][0] += 1
                    suspectsSummary[suspect][1][beneficiary][1] += allSuspects[day][suspect][beneficiary]
                else:
                    suspectsSummary[suspect][1][beneficiary] = [1, allSuspects[day][suspect][beneficiary]]

                    
            # print(suspectsSummary)

    # increase nWatchSuspectAsBeneficiary qnt index
    for day in allSuspects:
        for suspect in allSuspects[day]:
            for beneficiary in allSuspects[day][suspect]:
                if beneficiary in suspectsSummary:
                    # print("HERE MAN")
                    suspectsSummary[beneficiary][2] += 1
                    suspectsSummary[beneficiary][3] += allSuspects[day][suspect][beneficiary]
                    # print(suspectsSummary[beneficiary])
                    
            # print(suspectsSummary)
            # print("")
                
    # suspectsSummary = sorted(suspectsSummary.items(), key=lambda e: e[1][2])
    # order from the most observed to the least observed 
    suspectsSummary = OrderedDict(sorted(suspectsSummary.items(), key=lambda t: t[1][0], reverse=True))
    
    # print("Complete: ", len(suspectsSummary))
    # for suspect in suspectsSummary:
    #    print(suspect, suspectsSummary[suspect])
    
    # if suspect is observed only one time or is observed as many times as there are beneficiaries; 
    # is no longer a suspect
    for suspect in suspectsSummary:
        if suspectsSummary[suspect][0] > 1 and suspectsSummary[suspect][0] != suspectsSummary[suspect][2]:
            finalSuspects[suspect] = suspectsSummary[suspect]
            
    # print("With elimination: ", len(finalSuspects))
    # for suspect in finalSuspects:
        # print(suspect, finalSuspects[suspect])
    
    # print("")
    # print(finalSuspects)
    
    return finalSuspects
 
# compute a Guiltyindex (based to experimental observations) and print the node with more than .3
def edgesCount(finalSuspects, allNet):
    print("This are the main suspects!")
    edgesInCount = dict()
    edgesOutCount = dict()
    tradeEdgesInCount = dict()
    edgesOutPlayer = dict()

    
    activityDay = list()
    attackDay = list()
    tradeDay = list()
    
    inCount = 0
    outCount = 0
    tradeOutCount = 0
    tradeInCount = 0
    guiltyIndex = 0
    
    
    # count number of edge about all typology
    for suspect in finalSuspects:
        
        activityDay.clear()
        attackDay.clear()
        tradeDay.clear()
        
        edgesInCount.clear()
        edgesOutCount.clear()
        edgesOutPlayer.clear()
        tradeEdgesInCount.clear()
        
        for i in range(0, 3):
            edgesInCount['Attack'] = 0
            edgesInCount['Trade'] = 0
            edgesInCount['Message'] = 0
            edgesOutCount['Attack'] = 0
            edgesOutCount['Trade'] = 0
            edgesOutCount['Message'] = 0
            

        
        inCount = 0
        outCount = 0
        tradeOutCount = 0
        tradeInCount = 0
        guiltyIndex = 0
        
        # append a activity day of suspect
        for day in allNet:
            for vertex in allNet[day][4].vs:
                if str(vertex['id']) == suspect:
                    activityDay.append(str(day+1))
            

            for edge in allNet[day][4].es:
                if allNetwork[day][4].vs[edge.tuple[0]]['id'] == suspect:
                    if edge['Label'] in edgesOutCount:
                        edgesOutCount[edge['Label']] += 1
                    else:
                        edgesOutCount[edge['Label']] = 1
                    if edge['Label'] == 'Trade':
                        if allNetwork[day][4].vs[edge.tuple[1]]['id'] in edgesOutPlayer:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] += 1
                        else:
                            edgesOutPlayer[allNetwork[day][4].vs[edge.tuple[1]]['id']] = 1
                        if day not in tradeDay:
                            tradeDay.append(day + 1)
                if allNet[day][4].vs[edge.tuple[1]]['id'] == suspect:
                    if edge['Label'] == 'Trade':
                        if allNet[day][4].vs[edge.tuple[0]]['id'] in tradeEdgesInCount:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] += 1
                        else:
                            tradeEdgesInCount[allNetwork[day][4].vs[edge.tuple[0]]['id']] = 1
                    if edge['Label'] in edgesInCount:
                        edgesInCount[edge['Label']] += 1
                    else:
                        edgesInCount[edge['Label']] = 1
                    if edge['Label'] == 'Attack':
                        if day not in attackDay:
                            attackDay.append(day + 1)
        guiltyIndex = edgesOutCount['Trade']*((1/len(edgesOutPlayer)))*1.3
        # print("Trade out: ", edgesOutCount['Trade'])
        # print("People Served: ", len(edgesOutPlayer))
        # print(guiltyIndex)
        guiltyIndex -= (edgesInCount['Trade']*5)
        # print(guiltyIndex)
        guiltyIndex -= (edgesOutCount['Message'])*1
        # print(guiltyIndex)
        guiltyIndex += (edgesInCount['Message'])*0.3
        # print(guiltyIndex)
        guiltyIndex -= edgesOutCount['Attack']*0.4
        # print(guiltyIndex)
        guiltyIndex -= edgesInCount['Attack']*0.08
        # print(guiltyIndex)
        # print("DENOMINATOR: ", edgesInCount['Attack']+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']+edgesOutCount['Trade'])
        guiltyIndex = guiltyIndex / (edgesInCount['Attack']*0.08+edgesInCount['Trade']+edgesInCount['Message']+edgesOutCount['Attack']+edgesOutCount['Message']*2+edgesOutCount['Trade'])
        if guiltyIndex >= 0.3:
            print("GULTY INDEX: ", round(guiltyIndex, 4))
            print("SUSPECT: ", suspect)
            print("EDGES IN: ", edgesInCount)
            print("EDGES OUT: ", edgesOutCount)
            print("PLAYER SERVED: ", edgesOutPlayer)
            print("ACTIVITY DAYS: ", activityDay)
            print("DAYS OF ATTACKS SUFFERED: ", attackDay)
            print("DAYS OF TRADE DONE: ", tradeDay)
            print("PLAYER TRADE WITH SUSPECT: ", tradeEdgesInCount)
            print(" ")

## Communities Dynamics

#### Add communities label to vertex

In [0]:
def addCommunitiesLabel(allCompactNetwork, dayCommunity):
    print("DAY: ", (dayCommunity+1))
    communities = open(baseURL +  "Dataset/Communities/communities-2009-12-"+str(dayCommunity + 1)+".txt", "r")
    communityCont = 1
    nNodes = 0
    noMatchNodes = list()
    present = False
    nNodesPresent = 0
    smallCommunityId = -1
    communityAffiliates = dict()
    affiliatesNodes = list()
    
    nodesGraphNoMatch = list()
    vertexToDelete = list()
    
    for line in communities:
        # print("COMMUNITY NUMBER: ", cont)
        # print(line)
        line = line.split()
        # print(type(line[0]))
        for node in line:
            present = False
            nNodes += 1
            # allCompactNetwork.vs.select('id' = node)['Community'] = communityCont
            for vertex in allCompactNetwork.vs:
                if vertex['id'] == str(node):
                    if len(line) > 10:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = communityCont
                    else:
                        affiliatesNodes.append(vertex['id'])
                        vertex['Community'] = -communityCont
                    nNodesPresent += 1
                    present = True
            if not present:
                noMatchNodes.append(node)
       
        communityAffiliates[communityCont] = list(affiliatesNodes)
        communityCont += 1
        affiliatesNodes.clear()
        
    print("NUMBER OF NODES IN DAY TOTAL GRAPH: ", len(allCompactNetwork.vs))
    
    for vertex in allCompactNetwork.vs:
        if vertex['Community'] is None:
            nodesGraphNoMatch.append(vertex['id'])
            vertexToDelete.append(vertex.index)
            # print(vertex.index)
            # print(vertex)
    allCompactNetwork.delete_vertices(vertexToDelete)
        
    print("NUMBER OF NODES IN COMMUNITIES FILE: ", nNodes)
    print("NUMBER OF COMMUNITIES: ", communityCont)
    # print("NODES IN COMMUNITIES DAY FILE NO PRESENT IN DAY GRAPH: ", noMatchNodes)
    print("NUMBER OF NODES IN COMMUNITIES DAY FILE NL PRESENT IN DAY GRAPH: ", len(noMatchNodes))
    # print("NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", nodesGraphNoMatch)
    print("NUMBER OF NODES IN DAY GRAPH NO PRESENT IN COMMUNITIES DAY FILE: ", len(nodesGraphNoMatch))
    print("NUMBER OF NODES IN COMMUNITIES DAY FILE PRESENT IN DAY GRAPH: ", nNodesPresent)
    print("ALL DAY GRAPH AFTER REMOVING THE NO COMMUNITIES NODE: ", allCompactNetwork.summary())
    print("----------------------------------")
    return communityAffiliates

#### Communities War

In [0]:
def find_wars(dayGraph):
  #war = np.array(30,#community, #community)
  #for day in range(0,30)
    #for each attack edge
      #if att['community'] != def['community']
        #war[day, att['community'], def['community']] += 1
    #if war[day, att['community'], def['community']] > #min attack number to call it war
      #print('there was a war between A e B')
        

#### Community Centrality

In [0]:
def findDiplomats(dayGraph):
    print("DIPLOMATS DETECTION")
    edgeBet = dayGraph.edge_betweenness(weights = dayGraph.es['Weight'])
    # print(edgeBet)
    orderEdgeBet = np.argsort(edgeBet)[::-1]
    print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
      format("From", "To", "From Community", "To Community", "Edge Betweennees", "Weight"))
    for e in orderEdgeBet:
        if dayGraph.es[e]['Label'] == 'Message' and dayGraph.vs[dayGraph.es[e].tuple[0]]['Community'] != dayGraph.vs[dayGraph.es[e].tuple[1]]['Community']:
            print("{:>5}  {:>5}  {:>14}  {:>14}  {:>16}  {:>13}".
                format(dayGraph.vs[dayGraph.es[e].tuple[0]]['id'], dayGraph.vs[dayGraph.es[e].tuple[1]]['id'], 
                         dayGraph.vs[dayGraph.es[e].tuple[0]]['Community'], dayGraph.vs[dayGraph.es[e].tuple[1]]['Community'], 
                         round(edgeBet[e], 5), round(dayGraph.es[e]['Weight']), 0))

### Visualize Network
TODO

In [0]:
def VisualizeNetwork(allianceNet):
    print("USING Plotly Python Open Source Graphing Library")

## Main
From cell below we let's start all our analysis

In [32]:
#      TRY, IF YOU WANT!      #
 #############################
#userNumberTrend(allNetwork)  #
#edgeNumberTrend(allNetwork)  #
#tricksterAnalysis(allNetwork)#
 #############################
communityAffiliation = dict()

for i in range(0, 1):
    # print(allNetwork[i][5].summary())
    communityAffiliation[i] = addCommunitiesLabel(allNetwork[i][5], i)
    # addCommunitiesLabel(allNetwork[i][3], i)
    # print(allNetwork[i][5].summary())
print("")

# print(communityAffiliation)
    

findDiplomats(allNetwork[0][5])

# for vertex in allNetwork[i][5].vs:
#    print(vertex)
# print(allNetwork[1][5].vs[2])
# print(len(allNetwork[1][5].es))



DAY:  1


FileNotFoundError: ignored

Testing some community detection methods

In [0]:
#Print all the edges weights
# (allNetwork[0][3]).es["Weight"]
tryGraph = ig.Graph()
print(tryGraph.summary())
#Start Louvaign Detection
# getLouvainCommunities(allNetwork[0][3])

#This may crash your RAM
#getOptiModulaCommunities(allNetwork[0][3])
# getCommunityEdgeBetweenness(allNetwork[0][3])



IGRAPH U--- 0 0 -- 


reading the community file to compare results


In [0]:
#dictionary of #communities element, for each community there is an array with the id of the users of that community
commDict = dict()

#read the ground truth communities
filepath = baseURL +  'Dataset/Communities/communities-2009-12-1.txt'
with open(filepath) as fp:  
    line = fp.readline()
    cnt = 1
    while line:
        commDict[cnt]=line.strip();
        print("Community {}: {}".format(cnt, line.strip()))
        line = fp.readline()
        cnt += 1

for i in range(1,len(commDict)):
    commDict[i] = [int(s) for s in commDict[i].split(' ')]
  
print(commDict)

#number of nodes

nodes = 0;
for i in range(1,len(commDict)):
    nodes += len(commDict[i]);
print(nodes) #2665




Community 1: 1 2
Community 2: 2002 2132 5309 4025 2860 1108 3440 896
Community 3: 2026 8951 2939 1694 9638 2857 5126 1920 7638 1470 2085 7846 1445 6555 1228 3938 1317 6711 1725 8996 2264 4865 3369 825 3544 6977 2909 9598 2831 1155 5578 5736 2616 5395 2319 9487 3215 4444 775 479 4955 8357 1882 9822 7643 4490 1632 8896 9788 9948 1907 594 8156 7206 2290 5952 1008
Community 4: 1048
Community 5: 7595 6513 10085 8098 1640 10508 5658 2372 2622 7713 10568
Community 6: 1961 2600 11533 284 10460 7712 4245 2347 6690 3523 4298 862 6854 4519 11643 6759 5142 9067 11878 8321 2778 11858 6113 8149 10839 9523 2200 8644 9320 10161 11054 7682 8444 984 4878 5521 3092 8520 5913 10843 3281 7698 4992 10697 9192 8102 10242 9135 10812 1584 5657 8692 6118 4968 5402 3968 3089 11817
Community 7: 3806
Community 8: 4507 7617 2263 3709 2862 4415 10258 4165 1420 7303 9703
Community 9: 5234 787 1129 804 4510 965 725 4370 805 1156 4403 4499 750 4696 2427 2969 845 412 765 4585 5334 3008 3826 8806 3731 4811 5841 962 6822 

Testing differencies between csv and graphml

In [0]:
from igraph import *
import pandas as pd

import networkx as nx
import pandas as pd


#df = pd.read_csv(baseURL + "Dataset/CSV-Files/Messages-Network-CSV/messages-timestamped-2009-12-1.csv", header=None, usecols=[0,1,2], names=["time", "source", "target"]);
#Graphtype = nx.Graph()
mess_csv = nx.MultiDiGraph()
#mess_csv = nx.from_pandas_edgelist(df, edge_attr='time', create_using=Graphtype)
#print(mess_csv.edges())
#print(df);
#print(len(mess_csv.edges()));
#print(len(mess_csv.nodes()));

df = pd.read_csv(baseURL + "Dataset/CSV-Files/Trades-Network-CSV/trades-timestamped-2009-12-1.csv", header=None, usecols=[0,1,2], names=["time", "source", "target"]);
madonna = pd.read_csv(baseURL + "Dataset/CSV-Files/Trades-Network-CSV/trades-timestamped-2009-12-1.csv", header=None, usecols=[0,1,2]);
print(len(df));
Graphtype = nx.Graph()
cnt = 0;
for index, row in df.iterrows():
    cnt += 1;
    print(index, row)
    if (int(row['source']) == 1250):
      print('PORCAMADONNA')
    #print(row['time'], row['source'], row['target'])
    #Graphtype.add_edge(row['source'],row['target'], row['time']+'m');
    mess_csv.add_edge(int(row['source']),int(row['target']), key=int(row['time']), Label= "Message");
#for itr in list(mess_csv.edges):
        #print(itr)
print(cnt);
trade_csv = nx.from_pandas_edgelist(df, edge_attr='time', create_using=Graphtype)
#print(df);
print(len(mess_csv.edges()));
print(len(trade_csv.nodes()));
print(nx.info(mess_csv));
#g = Graph.Read_Ncol(baseURL + "Dataset/CSV-Files/Messages-Network-CSV/messages-timestamped-2009-12-1.csv", directed=True);
#g = nx.read_edgelist(baseURL + "Dataset/CSV-Files/Messages-Network-CSV/messages-timestamped-2009-12-1.csv", delimiter=',', nodetype=int, encoding="utf-8", data=(('time', int)));

'''
# create a multigraph for all nets type
messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()



messageSelf = 0

# number of graph (30 in our case)
nGraph = 30

# for every day is needed create total and alliance graph
for i in range(0, 1):
    print("Graph number: ", (i + 1))
    
    # for every day is needed reinitialize the "real" graphs (graph with edge label)
    
    realMessage = nx.MultiDiGraph()
    realTrade = nx.MultiDiGraph()
    
 
    messageSelf = 0
    
    # for every day is needed reinitialize the alliance graph and total graph
    allianceGraph = nx.MultiDiGraph()
    totalGraph = nx.MultiDiGraph()
    
    # read from Dataset/Graphml-File the correct file based on type and day
    
    messageGraph = mess_csv
    tradeGraph = trade_csv
    
    # copy xxxxGraph in realxxxx adding label to edges; keep attentio about key, it must be unique (we use old key + 
    # letter for define edge tipology) 
    for itr in list(messageGraph.edges(None, True, True)):
        print(itr)
    #for itr in messageGraph.edges_iter(None, True, True):
        if int(itr[0]) != int(itr[1]):
            realMessage.add_edge(int(itr[0]),int(itr[1]), key="m", Label= "Message", time = itr[2]['time'])
        else:
            messageSelf += 1
    
    #for itr in tradeGraph.edges_iter(None, True, True):
    for itr in list(tradeGraph.edges(None, True, True)):
        realTrade.add_edge(int(itr[0]),int(itr[1]), key="t", Label = "Trade", time = itr[2]['time'])


    
    
    # compose (a union) in one graph the needed graphs 
    allianceGraph = nx.compose(realMessage, realTrade)
    
  

    # print for check, i checked that sum of graphs is the equal to alliance graph and total graph (according to the case)
    # but i think is a better that YOU MAKE THE SAME; ONE MORE CONTROL IS BETTER!
   
    print("MESSAGE GRAPH: ", nx.info(messageGraph))
    print("NO SENSE SELF MESSAGE EDGE: ", messageSelf)
    print("TRADE GRAPH: ", nx.info(tradeGraph))
    print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
    print("TOTAL GRAPH: ", nx.info(totalGraph))
    print("") '''

9437
0 time      1259645047
source          7047
target          1481
Name: 0, dtype: int64
1 time      1259645031
source          1163
target          1309
Name: 1, dtype: int64
2 time      1259645025
source          6279
target          2975
Name: 2, dtype: int64
3 time      1259645024
source          7047
target          1481
Name: 3, dtype: int64
4 time      1259645025
source          2975
target          6279
Name: 4, dtype: int64
5 time      1259645022
source          2897
target          6626
Name: 5, dtype: int64
6 time      1259645016
source          2975
target          6279
Name: 6, dtype: int64
7 time      1259645016
source          6279
target          2975
Name: 7, dtype: int64
8 time      1259645008
source          2975
target          6279
Name: 8, dtype: int64
9 time      1259643651
source          5635
target          2200
Name: 9, dtype: int64
10 time      1259643665
source          5191
target          8139
Name: 10, dtype: int64
11 time      1259643666
source      

KeyboardInterrupt: ignored

# I prefer keep a bin cell rather than cry because i need already written code but deleted....
Of course in the final release this will be delete

In [0]:
# for libraries install
'''!sudo apt install build-essential python-dev libxml2 libxml2-dev zlib1g-dev
!sudo pip install cairocffi
!pip install python-igraph
!pip install louvain'''

attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, 1):
    totalGraph = ig.Graph()
    
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        if messageVertex.degree() != 0:
            allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            if messageVertex.degree() != 0:
                totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        # allianceGraph.add_edge(edge.tuple[0], edge.tuple[1], edgelabel='Message')
    # situa = 0
    # print("")
        

    

    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    
    print("")
    
    for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex.outdegree())
attackGraph = ig.Graph()
messageGraph = ig.Graph()
tradeGraph = ig.Graph()

totalPresent = False
alliancePresent = False

situa = 0

graphs = dict()
dayGraphs = list()

nGraph = 30

for i in range(0, nGraph):
    totalGraph = ig.Graph()
    allianceGraph = ig.Graph()
    print("Graph number: ", (i + 1))
    
    attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-"+str(i+1)+".graphml")
    for vertex in attackGraph.vs:
        totalGraph.add_vertices(vertex['id'])
        # print(vertex['id'] +", ", end="")
    # print("")
    print("ATTACK EDGE")
    for edge in attackGraph.es:
        # edge['edgelabel'] = "Attack"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(attackGraph.vs[edge.tuple[0]], attackGraph.vs[edge.tuple[1]], edgelabel='Attack') 
    situa = 0
    # print("")
    
    messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-"+str(i+1)+".graphml")
    for messageVertex in messageGraph.vs:
        allianceGraph.add_vertices(messageVertex['id'])
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if messageVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(messageVertex['id'])
        totalPresent = False
    
    print("MESSAGE EDGE")        
    for edge in messageGraph.es:
        # edge['edgelabel'] = "Message"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
        allianceGraph.add_edge(messageGraph.vs[edge.tuple[0]], messageGraph.vs[edge.tuple[1]], edgelabel='Message')
    # situa = 0
    # print("")
        
    tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-"+str(i+1)+".graphml")
    for tradeVertex in tradeGraph.vs:
        totalPresent = False
        for totalVertex in totalGraph.vs:
            if tradeVertex['id'] == totalVertex['name']:
                totalPresent = True
                break
        if not totalPresent:
            totalGraph.add_vertices(tradeVertex['id'])
        
    for tradeVertex in tradeGraph.vs:
        alliancePresent = False
        for allianceVertex in allianceGraph.vs:
            if tradeVertex['id'] == allianceVertex['name']:
                alliancePresent = True
                break
        if not alliancePresent:
            allianceGraph.add_vertices(tradeVertex['id'])
        
    
    print("TRADE EDGE")
    for edge in tradeGraph.es:
        # edge['edgelabel'] = "Trade"
        # print(situa, end = "")
        # print(", ", end = "")
        # situa += 1
        totalGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
        allianceGraph.add_edge(tradeGraph.vs[edge.tuple[0]], tradeGraph.vs[edge.tuple[1]], edgelabel='Trade')
    situa = 0
    # allianceGraph = messageGraph.union(tradeGraph)
    # totalGraph = allianceGraph.union(attackGraph)
    # print("")
    
    print("ATTACK GRAPH:", attackGraph.summary())
    print("MESSAGE GRAPH:", messageGraph.summary())
    print("TRADE GRAPH:", tradeGraph.summary())
    print("ALLIANCE GRAPH:", allianceGraph.summary())
    print("TOTAL GRAPH:", totalGraph.summary())
    totalGraph.save(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-"+str(i+1)+".graphml")
    allianceGraph.save(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    print("")
    






'''G = ig.Graph()
G = G.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 
G += G.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml") 
print(G)'''


# print(G.vs.attribute_names())
# print(G.es.attribute_names())
# print(G.degree())
# print(G.indegree())
# print(G.outdegree())

# totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")
allianceGraph = ig.Graph()
allianceGraph = allianceGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

totalGraph = ig.Graph()
totalGraph = totalGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-1.graphml")

attackGraph = ig.Graph()
attackGraph = attackGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml") 

tradeGraph = ig.Graph()
tradeGraph = tradeGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")

# print(totalGraph.degree())
# print(messageGraph.degree())
# print(attackGraph.degree())
# print(tradeGraph.degree())
print("AllianceGraph vertex with 0 degree")
for vertex in allianceGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id'] +", ", end = "")
print("")
print("")
print("TotalGraph vertex with 0 degree")
for vertex in totalGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")
print("")
print("")

for vertex in attackGraph.vs:
    if vertex.degree() == 0:
        print(vertex['id']+", ", end="")

print("")
print("")
ok = False
print("Attack Graph Check in Total Graph")
for vertex1 in attackGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 1")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Total Graph")
for vertex1 in messageGraph.vs:
    ok = False
    if vertex1['id'] == 'n2807':
        print("i am here 2")
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Total Graph")
for vertex1 in tradeGraph.vs:
    if vertex1['id'] == 'n2807':
        print("i am here 3")
    ok = False
    for vertex2 in totalGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")

print("")
print("")
ok = False
print("Message Graph Check in Alliance Graph")
for vertex1 in messageGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
        
print("")
print("")
ok = False
print("Trade Graph Check in Alliance Graph")
for vertex1 in tradeGraph.vs:
    ok = False
    for vertex2 in allianceGraph.vs:
        if vertex1['id'] == vertex2['name']:
            ok = True
            break
    if not ok:
        print("error")
    
print(allianceGraph.summary())
print(totalGraph.summary())

for vertex in totalGraph.vs:
    if vertex['id'] == 'n2807':
        print(vertex)
print("")
print("show in the graph")
for vertex in totalGraph.vs:
    if vertex['name'] == '9152':
        print(vertex)
    
print("In Total Graph")
for edge in totalGraph.es:
    if edge.tuple[0] == '9152' or edge.tuple[1] == '9152':
        print(edge)

print("In Message Graph")
for edge in messageGraph.es:
    if edge.tuple[0] == '9152':
        print(edge)
    if edge.tuple[1] == '9152':
        print(edge)
print("")
for vertex in attackGraph.vs:
    if vertex['id'] == '9152':
        print("attack")
        print(vertex)

for vertex in tradeGraph.vs:
    if vertex['id'] == '9152':
        print("trade")
        print(vertex)


for vertex in messageGraph.vs:
    if vertex['id'] == '9152':
        print("message")
        print(vertex)

        
list(attackGraph.vs)

import networkx as nx

messageGraph = nx.MultiDiGraph()
tradeGraph = nx.MultiDiGraph()
attackGraph = nx.MultiDiGraph()
allianceGraph = nx.MultiDiGraph()
totalGraph = nx.MultiDiGraph()

realMessage = nx.MultiDiGraph()
realTrade = nx.MultiDiGraph()
realAttack = nx.MultiDiGraph()

messageGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Messages-Network-Graphml/messages-timestamped-2009-12-1.graphml")
tradeGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Trades-Network-Graphml/trades-timestamped-2009-12-1.graphml")
attackGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Attacks-Network-Graphml/attacks-timestamped-2009-12-1.graphml")
# nx.info(messageGraph)

# print(messageGraph.nodes()[0])

# for edge in messageGraph.edges():   
#    messageGraph[edge[0]][edge[1]].update(label= "Message")

# nx.set_edge_attributes(messageGraph, "Label", "Message")
# nx.set_edge_attributes(tradeGraph, "Label", "Trade")
# nx.set_edge_attributes(attackGraph, "Label", "Attack")

# allianceGraph = nx.disjoint_union(messageGraph, tradeGraph)

# nx.set_edge_attributes(G, bb, 'betweenness')

# for itr in allianceGraph.edges_iter(None, True, True):
#        print(itr)



messageGraph.edges(data=True,keys=True)

for itr in messageGraph.edges_iter(None, True, True):
    realMessage.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"m", Label= "Message")
    
for itr in tradeGraph.edges_iter(None, True, True):
    realTrade.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"t", Label= "Trade")

for itr in attackGraph.edges_iter(None, True, True):
    realAttack.add_edge(int(itr[0]),int(itr[1]), key=str(itr[2])+"a", Label= "Attack")
    

allianceGraph = nx.compose(realMessage, realTrade)
totalGraph = nx.compose(allianceGraph, realAttack)
print("MESSAGE GRAPH: ", nx.info(messageGraph))
print("")
print("TRADE GRAPH: ", nx.info(tradeGraph))
print("")
print("ATTACK GRAPH: ", nx.info(attackGraph))
print("")
print("ALLIANCE GRAPH: ", nx.info(allianceGraph))
print("")
print("TOTAL GRAPH: ", nx.info(totalGraph))

# print(allianceGraph.degree())
print("")
print("")
# print(totalGraph.degree())

nx.write_graphml(realMessage, baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml")

messageGraph = ig.Graph()
messageGraph = messageGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Total-Network-Graphml/total-2009-12-30.graphml") 

print(messageGraph.summary())

# for edge in messageGraph.es:
#    print(edge)
    
#for vertex in messageGraph.vs:
#    print(vertex)

cont = 0
#print(allNetwork[0][4].indegree())
zeroIndegreeVertex = list()

for vertex in allNetwork[0][4].vs:
    if vertex.indegree() == 0:
        zeroIndegreeVertex.append(vertex)

# print(zeroIndegreeVertex)
suspected = dict()
key = ""
nEdges = 0
nTrades = 0

print("{:>7}{:>7}".format("From", "To"))
for vertex in zeroIndegreeVertex:
    nEdges = 0
    nTrades = 0
    for edge in allNetwork[0][4].es:
        if allNetwork[0][4].vs[edge.tuple[0]] == vertex:
            if edge['Label'] == 'Trade':
                nTrades += 1
            nEdges += 1
    if nEdges == nTrades:
        key = "from:" + allNetwork[0][4].vs[edge.tuple[0]]['id'] +" to:"+ allNetwork[0][4].vs[edge.tuple[1]]['id']
        if key not in suspected:
            suspected[key] = 1
        else:
            suspected[key] += 1

print(suspected)




nGraph = 30
realAllianceGraph = nx.DiGraph()
allianceGraph = nx.MultiDiGraph()

tryGraph = ig.Graph()
tryGraph = tryGraph.Read_GraphML(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-1.graphml")

edgesDict = dict()

tryCont = 0


tradeCount = 0
messageCount = 0


for i in range(0, 1):
    
    allianceGraph = nx.read_graphml(baseURL + "Dataset/Graphml-File/Alliance-Network-Graphml/alliance-2009-12-"+str(i+1)+".graphml")
    
    print("START GRAPH")
    print(nx.info(allianceGraph))
    print("")
    for itr in allianceGraph.edges_iter(None, True, True):
        if itr[0] in edgesDict:
                
            if itr[1] in edgesDict[itr[0]]:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]][0] += 1
                    tryCont += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]][1] += 1
                    tryCont += 1
                    tradeCount += 1
            else:
                if itr[3]['Label'] == 'Message':
                    edgesDict[itr[0]][itr[1]] = [1, 0]
                    tryCont += 1
                    messageCount += 1
                elif itr[3]['Label'] == 'Trade':
                    edgesDict[itr[0]][itr[1]] = [0, 1]
                    tryCont += 1
                    tradeCount += 1
                    
        else:
            edgesDict[itr[0]] = {}
            if itr[3]['Label'] == 'Message':
                edgesDict[itr[0]][itr[1]] = [1, 0]
                tryCont += 1
                messageCount += 1
            elif itr[3]['Label'] == 'Trade':
                edgesDict[itr[0]][itr[1]] = [0, 1]
                tryCont += 1
                tradeCount += 1
    suma = 0
    nStep = 0
    for fromVertex in edgesDict:
        for toVertex in edgesDict[fromVertex]:
            if  realAllianceGraph.has_edge(int(fromVertex), int(toVertex)):
                print("THIS IS A PROBLEM!")
            nStep += 1
            realAllianceGraph.add_edge(int(fromVertex), int(toVertex), Weight = sum(edgesDict[fromVertex][toVertex]))
            '''print("FROM VERTEX: ", fromVertex)
            print("VALUE: ", edgesDict[fromVertex])
            print("TO VERTEX: ", toVertex)
            print("VALUE: ", edgesDict[fromVertex][toVertex])
            print("SUM: ", sum(edgesDict[fromVertex][toVertex]))'''
            suma += sum(edgesDict[fromVertex][toVertex])
            # print("")
            
            #if itr[3]['Label'] = 'Message'
            #edgesDict[itr[0]][itr[1]][0] 
        
            
        # print(itr)

        
# for key, value in edgesDict:
    # print(key)
    # print("")

        
print("VERTEX NUMBER: ", len(edgesDict))
print("ALLIANCE GRAPH LENGHT: ", len(allianceGraph))



cont = 0
for vertex in tryGraph.vs:
    if vertex.outdegree() == 0:
        cont += 1
print("OUTDEGREE = 0 COUNT", cont)
print("")

print("END GRAPH")
print(nx.info(realAllianceGraph))
print("")
cont = 0
for itr in realAllianceGraph.edges_iter(None, True, True):
    cont += itr[2]['Weight']
    
    # print(itr)
print("OUR COUNT: ", cont)
print("CREATION COUNT: ", tryCont)
print("TRADE COUNT: ", tradeCount)
print("MESSAGE COUNT: ", messageCount)
print("TRADE COUNT + MESSAGE COUNT: ", (tradeCount+messageCount))
print("SUMA: ", suma)
print("NUMBER OF STEP: ", nStep)
print('-------------------------------')

    
